In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
from torch import optim
from pyprojroot import here
from torch.utils.data import DataLoader

from rxitect.data import SelfiesDataset, SmilesDataset
from rxitect.models import LSTMGenerator
from rxitect.tokenizers import SelfiesTokenizer, SmilesTokenizer, get_tokenizer

In [3]:
def smiles_tokenizer() -> SmilesTokenizer:
    test_vocabulary_filepath = here() / "tests/data/test_smiles_voc.txt"
    smiles_tokenizer = get_tokenizer("smiles", test_vocabulary_filepath, 100)
    return smiles_tokenizer

def smiles_dataloader(smiles_tokenizer) -> DataLoader:
    test_dataset_filepath = here() / "tests/data/test.smi"
    dataset = SmilesDataset(
        dataset_filepath=test_dataset_filepath, tokenizer=smiles_tokenizer
    )
    dataloader = DataLoader(
        dataset=dataset,
        batch_size=128,
        num_workers=1,
        shuffle=True,
        pin_memory=True,
        collate_fn=SmilesDataset.collate_fn,
    )
    return dataloader

In [4]:
net = LSTMGenerator(vocabulary_filepath="../tests/data/test_smiles_voc.txt", device="cuda")
dataloader = smiles_dataloader(net.tokenizer)

In [5]:
class Utils:
    def __init__(self):
        self.dev = "cuda"

utils = Utils()

lr = 1e-3
epochs = 10_000


optimizer = optim.Adam(net.parameters(), lr=lr)
for epoch in range(epochs):
    for i, batch in enumerate(dataloader):
        optimizer.zero_grad()
        loss_train = net.likelihood(batch.to(utils.dev))
        loss_train = -loss_train.mean()
        loss_train.backward()
        optimizer.step()
        if i % 10 == 0:
            seqs = net.sample(len(batch * 2))
            info = "Epoch: %d step: %d loss_train: %.3f" % (epoch, i, loss_train.item())

RuntimeError: gather(): Expected dtype int64 for index